In [1]:
import rclpy
import numpy as np
from rclpy.node import Node
from geometry_msgs.msg import PoseStamped
from math_methods import yaw_to_quaternion
from std_msgs.msg import String, Int8, UInt16MultiArray
import time
from geometry_msgs.msg import PoseWithCovarianceStamped

def create_coords_msg(x, y, th):
    msg = PoseStamped()
    msg.pose.position.x = x
    msg.pose.position.y = y
    msg.pose.orientation = yaw_to_quaternion(np.deg2rad(th))
    return msg

def create_stm_msg(string):
    msg = String()
    msg.data = string
    return msg

class GoalPublisher(Node):

    def __init__(self):


        self.side = 'blue'

        super().__init__("goal_pub")

        self.coords_publisher = self.create_publisher(
            PoseStamped, "robot2/nav/goal_pose", 10
        )
        self.commands_publisher = self.create_publisher(
            String, "robot2/stm/command", 10
        )

        self.nav_subscriber = self.create_subscription(
        Int8, "robot2/nav/feedback", self.nav_callback, 10
        )

        self.response_subscription = self.create_subscription(
        String,
        '/robot2/stm/response',
        self.response_callback,
        10)

        self.coords_subscription = self.create_subscription(
        PoseWithCovarianceStamped,
        '/robot2/filtered_coords',
        self.coords_callback,
        10)

        self.plants_clusters_subscription = self.create_subscription(
        UInt16MultiArray,
        '/plants',
        self.plants_clusters_callback,
        10)

        timer_period = 0.1  # seconds
        self.curr_tick = 0
        self.start_tick = 0
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.sent = False
        self.command = 0

        self.start_game = False
        self.moving = False
        self.grabbing = False

    def timer_callback(self):
        self.curr_tick += 1
        if self.side == 'blue':
            match self.command:
                case 0:
                    self.commands_publisher.publish(create_stm_msg("0 37"))
                case 1:
                    self.move_to(1.30, 0.7, -150)
                case 2:
                    self.open_gripper()

                # CONFIDENTIAL
                    
        elif self.side == 'yellow':
            match self.command:
                case 0:
                    self.commands_publisher.publish(create_stm_msg("0 37"))
                case 1:
                    self.move_to(1.30, 2.3, 30)
                case 2:
                    self.open_gripper()

                # CONFIDENTIAL
            
    def response_callback(self, msg):
        try:
            if int(msg.data[5]) == 0 and not self.start_game:
                self.start_game = True
                self.commands_publisher.publish(create_stm_msg("0 13 0"))  # init stepper
                self.command += 1
        except:
            pass

    def close_gripper(self):
        if not self.sent:
            self.start_tick = self.curr_tick
            self.commands_publisher.publish(create_stm_msg("0 10 0"))  # close gripper 0 OR 1
            self.sent = True
        if self.curr_tick - self.start_tick > 20:
            self.command += 1
            self.sent = False

    def open_gripper(self):
        if not self.sent:
            self.start_tick = self.curr_tick
            self.commands_publisher.publish(create_stm_msg("0 15 0"))  # close gripper 0 OR 1
            self.sent = True
        if self.curr_tick - self.start_tick > 20:
            self.command += 1
            self.sent = False

    def lift_gripper(self, height):
        if not self.sent:
            self.start_tick = self.curr_tick
            self.commands_publisher.publish(create_stm_msg(f"0 12 0 {height}")) # close gripper 0 OR 1
            self.sent = True
        if self.curr_tick - self.start_tick > 15:
            self.command += 1
            self.sent = False

    def nav_callback(self, msg):
        if msg.data == 1:
            self.moving = True
        else:
            self.moving = False

    def move_to(self, x, y, th):
        if not self.sent:
            msg = create_coords_msg(x, y, th)
            self.coords_publisher.publish(msg)
            self.moving = True
            self.sent = True
        if not self.moving:
            self.command += 1
            self.sent = False

    # self.available_clusters = [1, 1, 1, 1, 1, 1]

    # if self.side == 'blue':
    #     self.available_planters = [1, 0, 1, 0, 0, 1]
    #     self.available_zones = [1, 0, 1, 0, 1, 0]
    # else:
    #     self.available_planters = [0, 1, 0, 1, 1, 0]
    #     self.available_zones = [0, 1, 0, 1, 0, 1]
    
    # self.x = 0
    # self.y = 0
    # self.z = 0

    def coords_callback(self, msg):
        self.x = msg.pose.pose.position.x
        self.y = msg.pose.pose.position.y
        self.z = msg.pose.pose.position.z
        self.get_logger().info('COORDINATES are: %f, %f, %f' % \
                               (msg.pose.pose.position.x, msg.pose.pose.position.y, msg.pose.pose.position.z))

    
    def plants_clusters_callback(self, msg):
        self.available_clusters = msg.data
        self.get_logger().info('CLUSTERS STATES are: %d, %d, %d, %d, %d, %d' % \
                               (msg.data[0], msg.data[1], msg.data[2], msg.data[3], msg.data[4], msg.data[5]))


    def move_to_closest_cluster(self):
        
        clusters_coords = ((0.7, 1.0), (1.3, 1.0), (1.5, 1.5), (1.3, 2.0), (0.7, 2.0), (0.5, 1.5))
        min_dist = 100000
        min_ind = 1

        curr_x = self.x
        curr_y = self.y

        available = False

        for i in range(len(clusters_coords)):
            if self.available_clusters[i]:
                available = True
                dist = np.sqrt((curr_x - clusters_coords[i][0])**2 + (curr_y - clusters_coords[i][1])**)
                if dist < min_dist:
                    min_dist = dist
                    min_ind = i

        if not available:
            return False
        
        self.available_clusters[min_ind] = 0
        r = 0.2
        center = clusters_coords[min_ind]
        angle = np.arctan2(center[1] - curr_y, center[0] - curr_x) #mb need to add some rotation
        x_goal = center[0] - r * np.cos(angle)
        y_goal = center[0] - r * np.sin(angle)

        self.move_to(x_goal, y_goal, angle - 120)


    # def move_to_closest_planter(self):

    #     offset = 0.10
    #     planters_coords = ((0.61, offset, 30), (1.39, offset, 30), (1.39, 3 - offset, 210), \
    #     (0.61, 3 - offset, 210), (offset, 2.24, -60), (offset, 0.76, -60))
    #     for i in range(len(planters_coords)):
    #         if self.available_planters[i]:
    #     curr_x = self.x
    #     curr_y = self.y
        
    #     min_dist = 10000
    #     min_ind = 1

    #     available = False

    #     for i in range(len(planters_coords)): # first of all, check planters
    #         if self.available_planters[i]:
    #             available = True
    #             dist = np.sqrt((curr_x - planters_coords[i][0])**2 + (curr_y - planters_coords[i][1])**2)
    #             if dist < min_dist:
    #                 min_dist = dist
    #                 min_ind = i

    #     if available:
    #         self.available_planters[min_ind] = 0
    #         self.move_to(planters_coords[min_ind][0], planters_coords[min_ind][1], planters_coords[min_ind][2])

    #     else: # if could not find available planters, drop plants to drop zones
    #         zones_coords = ((0.23, 0.23, 30), (1.0, 0.23, 30), (1.77, 0.23, 30), \
    #    (1.77, 2.77, 210), (1.0, 2.77, 210), (0.23, 2.77, 210))
    #         for i in range(len(zones_coords)): # first of all, check planters
    #             if self.available_zones[i]:
    #                 dist = np.sqrt((curr_x - zones_coords[i][0])**2 + (curr_y - zones_coords[i][1])**2)
    #                 if dist < min_dist:
    #                     min_dist = dist
    #                     min_ind = i

    #         self.move_to(zones_coords[min_ind][0], zones_coords[min_ind][1], zones_coords[min_ind][2])

    # def move_to(self, x, y, th):
        # msg = create_coords_msg(x, y, th)
        # self.coords_publisher.publish(msg)

    def wiggle(self):
        self.get_logger().info("Calibration (wiggle-wiggle-wiggle)")

        self.commands_publisher.publish(
            create_stm_msg("0 4 -0.177 -0.014 -0.03")
        )  # move along x
        time.sleep(0.2)

        for _ in range(2):
            self.commands_publisher.publish(
                create_stm_msg("0 4 0.122 -0.186 -0.03")
            )  # move along x
            time.sleep(0.4)
            self.commands_publisher.publish(
                create_stm_msg("0 4 -0.177 -0.014 -0.03")
            )  # move along XY diagonal
            time.sleep(0.4)
        self.commands_publisher.publish(
            create_stm_msg("0 4 0.122 -0.186 -0.03")
        )  # move along x
        time.sleep(0.2)
        self.commands_publisher.publish(create_stm_msg("0 4 0 0 0"))  # stop
        time.sleep(0.1)

    # def move_forward_a_bit(self):
    #     self.get_logger().info("Moving forward a bit")
    #     self.commands_publisher.publish(
    #         create_stm_msg("0 4 -0.125 -0.215 -0.06")
    #     )  # move along x
    #     time.sleep(0.4)
    #     self.commands_publisher.publish(create_stm_msg("0 4 0 0 0"))  # stop
    #     time.sleep(0.1)

    # def init_grippers(self):
    #     self.get_logger().info("Initializing grippers")
    #     self.commands_publisher.publish(create_stm_msg("0 13 0"))  # init stepper
    #     time.sleep(0.2)
    #     self.commands_publisher.publish(create_stm_msg("0 13 1"))  # init stepper
    #     time.sleep(0.2)
    #     self.commands_publisher.publish(create_stm_msg("0 10 0"))  # close gripper
    #     time.sleep(0.2)
    #     self.commands_publisher.publish(create_stm_msg("0 10 1"))  # close gripper
    #     time.sleep(1)
    #     self.commands_publisher.publish(create_stm_msg("0 15 0"))  # open gripper
    #     time.sleep(0.2)
    #     self.commands_publisher.publish(create_stm_msg("0 15 1"))  # open gripper
    #     time.sleep(0.2)
    #     self.commands_publisher.publish(create_stm_msg("0 11 2 0"))  # fold manipulators
    #     time.sleep(2)
    #     self.commands_publisher.publish(create_stm_msg("0 12 1 18350"))  # move stepper 1
    #     time.sleep(3)

    # def move_in_local_coordiates(self, x, y):
    #     coords = np.array([[x], [y]])
    #     v = 0.2
    #     coeff = 1.5
    #     d = np.sqrt(np.sum(coords**2))
    #     t = d / v
    #     v_components = coords / t
    #     R = np.array([[np.cos(2*np.pi/3), np.sin(2*np.pi/3)],
    #                 [-np.sin(2*np.pi/3), np.cos(2*np.pi/3)]])
    #     v_stm = R @ v_components
    #     message = '0 4 ' + "%.2f" % v_stm[0][0] + ' ' + "%.2f" % v_stm[1][0] + ' 0.00'
    #     self.commands_publisher.publish(create_stm_msg(message))
    #     t *= coeff
    #     time.sleep(t)
    #     self.commands_publisher.publish(create_stm_msg("0 4 0.0 0.0 0.0"))

    # STATE MACHINE VERSION
        
    def move_in_local_coordiates(self, x, y):

        if not self.sent:
            self.start_tick = self.curr_tick
            coords = np.array([[x], [y]])
            v = 0.2
            coeff = 1.5
            d = np.sqrt(np.sum(coords**2))
            t = d / v
            v_components = coords / t
            R = np.array([[np.cos(2*np.pi/3), np.sin(2*np.pi/3)],
                        [-np.sin(2*np.pi/3), np.cos(2*np.pi/3)]])
            v_stm = R @ v_components
            message = '0 4 ' + "%.2f" % v_stm[0][0] + ' ' + "%.2f" % v_stm[1][0] + ' 0.00'
            self.commands_publisher.publish(create_stm_msg(message))
            t *= coeff
            self.sent = True
        else:
            coords = np.array([[x], [y]])
            v = 0.2
            coeff = 1.5
            d = np.sqrt(np.sum(coords**2))
            t = d / v
        if self.curr_tick - self.start_tick > int(t*10):
            self.commands_publisher.publish(create_stm_msg("0 4 0.0 0.0 0.0"))
            self.command += 1
            self.sent = False


In [2]:
try:
    rclpy.init(args=None)
    goal_pub = GoalPublisher()
    rclpy.spin(goal_pub)
    
except KeyboardInterrupt:

    try:
        goal_pub.destroy_node()
    except:
        print("Could not destroy the node")

    try:
        rclpy.shutdown()
    except:
        print("Could not shut rclpy down")

In [1]:
# SOLAR PANEL ALIIGNMENT WITH CAMERA

In [ ]:
import cv2
import numpy as np
import time
import cv2.aruco

In [58]:
video = cv2.VideoCapture(0)
video.set(cv2.CAP_PROP_AUTOFOCUS, 0)
# video.set(cv2.CAP_PROP_AUTO_EXPOSURE, -6.0)
# video.set(cv2.CAP_PROP_SETTINGS, 0)

True

In [57]:
video.release()

In [61]:
parameters = cv2.aruco.DetectorParameters()

mtx = np.array([[511.6755209 ,   0.        , 318.33463582],
                [  0.        , 509.57149594, 260.78914829],
                [  0.        ,   0.        ,   1.        ]])

dist = np.array([[ 9.18064536e-02, -2.18028591e-01,  2.94281069e-06,
         6.06424752e-04,  7.83251200e-02]])

aruco_dict_type = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)

def pose_esitmation(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.aruco_dict = aruco_dict_type
    parameters.aprilTagMinWhiteBlackDiff = 1
    # parameters.aprilTagQuadSigma = 10.0
    # parameters.aprilTagDeglitch = 1
    # parameters.aprilTagQuadDecimate = 1
    # parameters.aprilTagMinClusterPixels = 1
    # parameters.adaptiveThreshConstant = 1
    # parameters.adaptiveThreshWinSizeMax = 50
    # parameters.adaptiveThreshWinSizeMin = 20
    
    
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters)

        # If markers are detected
    if len(corners) > 0:
        for i in range(0, len(ids)):
            # Estimate pose of each marker and return the values rvec and tvec---(different from those of camera coefficients)
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.037, mtx, dist)
            # Draw a square around the markers
            # cv2.aruco.drawDetectedMarkers(frame, corners) 

            # Draw Axis
            # cv2.drawFrameAxes(frame, mtx, dist, rvec, tvec, 0.01)

            aruco_rot_matrix = cv2.Rodrigues(rvec[0][0])[0]
            phi = np.arctan2(aruco_rot_matrix[1][0], aruco_rot_matrix[0][0])

            # if -2 < phi < -1:
            #     print("Blue")
            # elif 1 < phi < 2:
            #     print("Yellow")
            # else:
            #     print("Not stated")

            # aruco_trans_matrix = np.zeros((4, 4), dtype='float')
            # aruco_trans_matrix
                
            # print(tvec[0][0])
            disp_x = tvec[0][0][0]
            disp_y = tvec[0][0][1]
            print(disp_x, disp_y)
            if ((disp_x**2 + disp_y**2)**0.5) > 0.04:
                goal_pub.move_in_local_coordiates(-disp_y/4, -disp_x/4)
            else:
                goal_pub.commands_publisher.publish(create_stm_msg("0 4 0 0 0"))

    return rvec, tvec

def check_panel_orientation(rvec):
    aruco_rot_matrix = cv2.Rodrigues(rvec[0][0])[0]
    phi = np.arctan2(aruco_rot_matrix[1][0], aruco_rot_matrix[0][0])
    if -2 < phi < -1:
        print("Blue")
    elif 1 < phi < 2:
        print("Yellow")
    else:
        print("Not stated")

def align_with_solar_panel():

    cv2.aruco_dict = aruco_dict_type
    parameters.aprilTagMinWhiteBlackDiff = 1
    parameters.aprilTagMinWhiteBlackDiff = 1
    # parameters.aprilTagQuadSigma = 10.0
    # parameters.aprilTagDeglitch = 1
    # parameters.aprilTagQuadDecimate = 1
    # parameters.aprilTagMinClusterPixels = 1
    # parameters.adaptiveThreshConstant = 1
    # parameters.adaptiveThreshWinSizeMax = 50
    # parameters.adaptiveThreshWinSizeMin = 20

    ret, frame = video.read()
    if not ret:
        print("Frame aquisition error")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict, parameters=parameters)

    # if nothing is detected, move around a bit
    while len(corners) == 0:
        print("No markers detected")
        rand_x = np.random.uniform(-0.02, 0.02)
        rand_y = np.random.uniform(-0.02, 0.02)
        goal_pub.move_in_local_coordiates(rand_x, rand_y)
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict, parameters=parameters)
    
    # for i in range(0, len(ids)):
    rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[0], 0.037, mtx, dist)
    disp_x = tvec[0][0][0]
    disp_y = tvec[0][0][1]

    # if ((disp_x**2 + disp_y**2)**0.5) > 0.04:
    #     goal_pub.move_in_local_coordiates(-disp_y/4, -disp_x/4)
    # else:
    #     goal_pub.commands_publisher.publish(create_stm_msg("0 4 0 0 0"))
    #     time.sleep(0.1)

    # while ((disp_x**2 + disp_y**2)**0.5) > 0.03:
    while True:
        print("Moving")
        goal_pub.move_in_local_coordiates(-disp_y/5, -disp_x/5)
        ret, frame = video.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict, parameters=parameters)
        if len(corners) > 0:
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[0], 0.037, mtx, dist)
            disp_x = tvec[0][0][0]
            disp_y = tvec[0][0][1]
    print("Reached")

SyntaxError: 'break' outside loop (1633957986.py, line 94)